In [1]:
import pandas as pd 
import numpy as np 

In [2]:
dataframe = pd.read_csv("iris.data")
print(dataframe.shape)
# chuyển data về dạng numpy 
print(dataframe.head(2))

(149, 5)
   5.1  3.5  1.4  0.2  Iris-setosa
0  4.9  3.0  1.4  0.2  Iris-setosa
1  4.7  3.2  1.3  0.2  Iris-setosa


In [3]:
# chuyển data về dạng numpy 
data = dataframe.values
print(data.shape)

(149, 5)


In [4]:
X = data[:,0:-1]
print(X.shape)
print(X[:5])

(149, 4)
[[4.9 3.0 1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.0 3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]]


In [5]:
y = data[:,-1]
y.ravel()
print(y.shape)
print(y[:10])

(149,)
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(\
    X, y, test_size=0.3, random_state=42)

print('Train:')
print(X_train.shape)
print(y_train.shape)
print('Test:')
print(X_test.shape)
print(y_test.shape)

Train:
(104, 4)
(104,)
Test:
(45, 4)
(45,)


In [7]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
# training process
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [8]:
y_pred = model.predict(X_test)

In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        19
Iris-versicolor       0.91      0.77      0.83        13
 Iris-virginica       0.80      0.92      0.86        13

       accuracy                           0.91        45
      macro avg       0.90      0.90      0.90        45
   weighted avg       0.92      0.91      0.91        45

